# Lending Case Study

In [ ]:
## Importing all necessary libs 

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_rows', 99999)

In [ ]:
df = pd.read_csv('loan.csv')

In [ ]:
df.head()

In [ ]:
print("Shape: ", df.shape)
df.columns[df.isna().sum() > 0]

In [ ]:
## Dropping all columns that contain NULL values

df.dropna(axis=1, how='all', inplace=True)
print('Shape: ', df.shape)
df.columns[df.isna().sum() > 0]

In [ ]:
print(df.isna().sum()/len(df.columns))

## Columns containing 0 null values
print(df.columns[df.isna().sum() > 0])

### There are 14 more columns that still contains missing values.

Filling in the missing values in the columns with the mode/mean/median of the columns can be done.
Since there are 14 columns, 
Lets identify which of the columns would be needed for the analysis and which of the columns would not be needed for the analysis.

In [ ]:
print(df.columns)

### columns with all unique values can be dropped as they do not provide any information

In [ ]:
## Identifying columns with all unique values
print("Shape: ",df.shape)
# print(df.nunique())
df_unique = (df.nunique() == 1) 
df_unique = df_unique[df_unique == True]
print(df_unique)


## Dropping columns that contain only 1 unique value
df.drop(df_unique.index, axis=1, inplace=True)
df.shape

In [ ]:
def print_unique_and_null_values(df_x):
    print(df_x.shape)
    # for col in df.columns:
    #     print(col, df[col].nunique(), df[col].isna().sum())
    na_sum = df_x.isna().sum()
    n_unique = df_x.nunique()
    na_percentage = ((na_sum/len(df_x)) * 100).astype(int).apply(str) + '%'

    na_sum_df = na_sum.reset_index()
    n_unique_df = n_unique.reset_index()
    na_percentage_df = na_percentage.reset_index()

    n_unique_df.columns = ['Column', 'Unique Values']
    na_sum_df.columns = ['Column', 'Missing Values']
    na_percentage_df.columns = ['Column', 'Missing Percentage']

    merged_df = pd.merge( n_unique_df,na_sum_df, on='Column')
    merged_df = pd.merge(merged_df, na_percentage_df, on='Column')
    print(merged_df)

In [ ]:
print("Shape: \n", df.shape)
print_unique_and_null_values(df)

### Columns that are not required for analysis
(based on the value present in the cells and data dictionary)

In [ ]:
def remove_columns(df, columns_to_remove):
    df_x = df.copy()
    df_x.drop(columns_to_remove, axis=1, inplace=True)
    return df_x


# Columns that are not required for analysis based on the present value and data dictionary
irrelevant_columns = np.array([
    # 'id',
    'member_id',
    'url',
    'desc',
    'title',
    'zip_code',
    'addr_state',
])

print("irrelevant_columns.shape: ", irrelevant_columns.shape)

df_1 = remove_columns(df, irrelevant_columns)

print("Shape: \n", df_1.shape)
print_unique_and_null_values(df_1)

In [ ]:
# Columns that are post-loan approval and not required for analysis
post_approval_columns = np.array([
    'emp_title', 

    'funded_amnt',

    'issue_d', 
    'delinq_2yrs', 
    
    'mths_since_last_delinq',
    'mths_since_last_record',

    'revol_bal',

    "out_prncp",
    "out_prncp_inv",

    'total_pymnt',
    'total_pymnt_inv',
    'total_rec_prncp',
    'total_rec_int',
    'total_rec_late_fee',

    'recoveries',
    'collection_recovery_fee', 

    'last_pymnt_d',
    'last_pymnt_amnt', 
    'next_pymnt_d', 
    'last_credit_pull_d'
    # 'pub_rec_bankruptcies'
    ])

print("post_approval_columns.shape: ",post_approval_columns.shape)

df_1 = remove_columns(df_1, post_approval_columns)

print("Shape: \n", df_1.shape)
print_unique_and_null_values(df_1)


In [ ]:
# removing outliers from annual_income
sns.boxplot(df_1['annual_inc'])

In [ ]:
quantile_info = df_1.annual_inc.quantile([0.5, 0.75,0.90, 0.95, 0.97,0.98, 0.99,1.0])
quantile_info

In [ ]:
per_95_annual_inc = df_1['annual_inc'].quantile(0.99)
df_1 = df_1[df_1.annual_inc <= per_95_annual_inc]

# Data Standardization

In [ ]:
## emp_length column

emp_length_default = df_1['emp_length'].mode()[0]

df_1['emp_length'].value_counts()
df_1['emp_length'].fillna(emp_length_default, inplace=True)

print_unique_and_null_values(df_1)

In [ ]:
## revol_util column
df_1['revol_util'].isna().sum()

## since the number of missing values is very low, we can drop the rows with missing values
df_1.dropna(axis=0, subset=['revol_util'], inplace=True)

print_unique_and_null_values(df_1)

In [ ]:
## pub_rec_bankruptcies column

df_1['pub_rec_bankruptcies'].value_counts()

## records with loan_status == 'Current', should not be considered for analysis, since the loan_status is not final
df_1= df_1[ df_1['loan_status'] != 'Current' ]


## records with loan_status == 'Fully Paid', probabily have pub_rec_bankruptcies == 0
# df_1['pub_rec_bankruptcies'] = np.where(df_1['loan_status'] == 'Fully Paid', 0, df_1['pub_rec_bankruptcies'])

print_unique_and_null_values(df_1)

In [ ]:
## pub_rec_bankruptcies column
df_1_pub_rec_bankruptcies_na = df_1[ df_1['pub_rec_bankruptcies'].isna()]

count = df_1['loan_status'].value_counts()
na_len =  df_1_pub_rec_bankruptcies_na['loan_status'].value_counts()

print(count)
print(na_len)
print('\n',(na_len/count)*100)

In [ ]:
## since the ratio of missing values is very low, we can drop the rows with missing values
df_1.dropna(axis=0, subset=['pub_rec_bankruptcies'], inplace=True)

print_unique_and_null_values(df_1)

In [ ]:
df_1.info()

In [ ]:
df_2 = df_1.copy()

df_2['term'] = df_2['term'].str.extract('(\d+)').astype(int)
df_2['int_rate'] = df_2['int_rate'].str.extract('(\d+.\d+)').astype(float)
df_2['emp_length'] = df_2['emp_length'].str.extract('(\d+)').astype(int)


def standardize_dates(date_str):
    if '/' in date_str:
        return pd.to_datetime(date_str, format='%d/%m/%Y', errors='coerce').strftime('%Y-%m-%d')
    else:
        return pd.to_datetime(date_str, format='%b-%y', errors='coerce').strftime('%Y-%m-%d')

# df_2['earliest_cr_line'] = df_2['earliest_cr_line'].apply(lambda x: standardize_dates(x))

# df_2['earliest_cr_line'] = pd.to_datetime(df_2['earliest_cr_line'], format='%Y-%m-%d', errors='coerce')


print(df_2.info())
(df_2.head())


#  Analysis

##  Univariate Analysis

### Categorical data

In [ ]:
df_3 = df_2.copy()

df_3_charged_off = df_3[ df_3['loan_status'] == 'Charged Off']
df_3_fully_paid = df_3[ df_3['loan_status'] == 'Fully Paid']

In [ ]:
# print(df_3['loan_status'].value_counts())
# df_3['loan_status'].value_counts().plot(kind='bar')


sns.countplot(x='loan_status', data=df_2)
plt.title('Count of Items in Each Status')
plt.xlabel('Loan Status')
plt.ylabel('Count')
plt.show()

In [ ]:
# sns.countplot( x='grade', order = ['A', 'B', 'C', 'D', 'E', 'F', 'G'], data=df_3_charged_off)
# plt.title('Distribution of Charged Off Loans by Grade')
# plt.xlabel('Category')
# plt.ylabel('Value')
# plt.show()

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 6))  # 1 row, 2 columns

axes[0].set_title('Distribution of Fully Paid Loans by Grade')
axes[1].set_title('Distribution of Charged Off Loans by Grade')

sns.countplot( x='grade', order = ['A', 'B', 'C', 'D', 'E', 'F', 'G'], data=df_3_fully_paid, ax=axes[0])
sns.countplot( x='grade', order = ['A', 'B', 'C', 'D', 'E', 'F', 'G'], data=df_3_charged_off, ax=axes[1])

plt.show()

In [ ]:
fig, axes = plt.subplots(2,2, figsize=(10, 6))  # 1 row, 2 columns

axes[0][0].set_title('Distribution of Fully Paid Loans by Grade')
axes[0][1].set_title('Distribution of Charged Off Loans by Grade')

## by subgrades
axes[1][0].set_title('Distribution of Fully Paid Loans by Subgrade')
axes[1][1].set_title('Distribution of Charged Off Loans by Subgrade')

sns.countplot( x='grade', order = ['A', 'B', 'C', 'D', 'E', 'F', 'G'], data=df_3_fully_paid, ax=axes[0][0])
sns.countplot( x='grade', order = ['A', 'B', 'C', 'D', 'E', 'F', 'G'], data=df_3_charged_off, ax=axes[0][1])

sns.countplot( x='sub_grade', order = ['A1', 'A2', 'A3', 'A4', 'A5', 'B1', 'B2', 'B3', 'B4', 'B5', 'C1', 'C2', 'C3', 'C4', 'C5', 'D1', 'D2', 'D3', 'D4', 'D5', 'E1', 'E2', 'E3', 'E4', 'E5', 'F1', 'F2', 'F3', 'F4', 'F5', 'G1', 'G2', 'G3', 'G4', 'G5'], data=df_3_fully_paid, ax=axes[1][0])
sns.countplot( x='sub_grade', order = ['A1', 'A2', 'A3', 'A4', 'A5', 'B1', 'B2', 'B3', 'B4', 'B5', 'C1', 'C2', 'C3', 'C4', 'C5', 'D1', 'D2', 'D3', 'D4', 'D5', 'E1', 'E2', 'E3', 'E4', 'E5', 'F1', 'F2', 'F3', 'F4', 'F5', 'G1', 'G2', 'G3', 'G4', 'G5'], data=df_3_charged_off, ax=axes[1][1])

plt.show()

In [ ]:
df_3['sub_grade'] = pd.to_numeric(df_3.sub_grade.apply(lambda x : x[-1]))
df_3_charged_off = df_3[ df_3['loan_status'] == 'Charged Off']
df_3_fully_paid = df_3[ df_3['loan_status'] == 'Fully Paid']

In [ ]:
fig, axes = plt.subplots( figsize=(12, 6))  # 1 row, 2 columns

axes.set_title('Distribution of Charged Off Loans by Grade')
sns.countplot( x='grade', order = ['A', 'B', 'C', 'D', 'E', 'F', 'G'], hue = 'sub_grade', data=df_3_charged_off )
plt.show()

In [ ]:
print(df_3_charged_off['purpose'].value_counts())

ig, ax = plt.subplots(figsize = (10,8))
sns.countplot(x ='purpose', data=df_3_charged_off)
ax.set_xticklabels(ax.get_xticklabels(), rotation=40, ha="right")
plt.show()

In [ ]:
# since the number of records for 'debt_consolidation' is very high,let try using a log scale
ig, ax = plt.subplots(figsize = (10,8))
sns.countplot(y ='purpose', data=df_3_charged_off)
ax.set(xscale = 'log')
plt.show()

In [ ]:
df_3.info()

In [ ]:
print(df_3['home_ownership'].value_counts())

fig, axes = plt.subplots( figsize=(12, 6))
axes.set_title('Distribution of Charged Off Loans by Home Ownership')
sns.countplot( x='home_ownership', order=[ 'RENT', 'MORTGAGE', 'OWN', 'OTHER'], data=df_3_charged_off)
axes.set(yscale = 'log')  # since the count of 'OTHER' is very low
plt.show() 

In [ ]:
print(df_3['home_ownership'].value_counts())

fig, axes = plt.subplots( figsize=(12, 6))
axes.set_title('Distribution of Charged Off Loans by Home Ownership')
sns.countplot( x='home_ownership', order=[ 'RENT', 'MORTGAGE', 'OWN', 'OTHER'], data=df_3_charged_off)
axes.set(yscale = 'log')  # since the count of 'OTHER' is very low
plt.show() 

In [ ]:
print(df_3['verification_status'].value_counts())

fig, axes = plt.subplots( figsize=(12, 6))
axes.set_title('Distribution of Charged Off Loans by Verification Status')
sns.countplot( x='verification_status', order=['Not Verified', 'Source Verified', 'Verified'], data=df_3_charged_off)
plt.show() 

In [ ]:
df_3['revol_util'] = df_3['revol_util'].str.extract('(\d+.\d+)').astype(float)

df_3['revol_util_groups'] = pd.cut(df_3['revol_util'], bins=5,precision =0,labels=['0-20','20-40','40-60','60-80','80-100'])

df_3_charged_off = df_3[ df_3['loan_status'] == 'Charged Off']
df_3_fully_paid = df_3[ df_3['loan_status'] == 'Fully Paid']

In [ ]:
fig, axes = plt.subplots( figsize=(12, 6))
axes.set_title('Distribution of Charged Off Loans by Revolving Utilization')
sns.countplot( x='revol_util_groups', data=df_3_charged_off)
plt.show()

### Numerical data

In [ ]:
df_3.info()

In [ ]:
print(df_3_charged_off['loan_amnt'].describe())

fig, axes = plt.subplots( figsize=(12, 6))
axes.set_title('Distribution of Charged Off Loans by Loan Amount')
sns.histplot( x='loan_amnt', data=df_3_charged_off, bins=20)
plt.show()

In [ ]:
print(df_3_charged_off['term'].value_counts())

fig, axes = plt.subplots( figsize=(12, 6))
axes.set_title('Distribution of Charged Off Loans by Term')
sns.countplot( x='term', data=df_3_charged_off)
plt.show()

In [ ]:
print(df_3['int_rate'].describe())
df_3['int_rate_groups'] = pd.cut(df_3['int_rate'], bins=5,precision =0,labels=['5%-9%','9%-13%','13%-17%','17%-21%','21%-24%'])

df_3_charged_off = df_3[ df_3['loan_status'] == 'Charged Off']
df_3_fully_paid = df_3[ df_3['loan_status'] == 'Fully Paid']

In [ ]:

fig, axes = plt.subplots( figsize=(12, 6))
axes.set_title('Distribution of Charged Off Loans by Interest Rate')
sns.countplot( x='int_rate_groups', data=df_3_charged_off)
plt.show()

In [ ]:
# print(df_3_charged_off['int_rate'].describe())
# df_3['int_rate_groups'].value_counts()

# print(df_3['int_rate'].describe())
bin_4 = df_3.copy()
bin_4['int_rate_groups'] = pd.cut(df_3_charged_off['int_rate'], bins=4,precision =0,labels=['5%-10%','10%-15%','15%-20%','20%-25%'])

fig, axes = plt.subplots( figsize=(12, 6))
axes.set_title('Distribution of Charged Off Loans by Interest Rate')
sns.countplot( x='int_rate_groups', data=bin_4)
plt.show()

In [ ]:
bin_10 = df_3.copy()
bin_10['int_rate_groups'] = pd.cut(df_3_charged_off['int_rate'], bins=10,precision =0,labels=[
    '5%-7%','7%-9%','9%-11%','11%-13%','13%-15%','15%-17%','17%-19%','19%-21%','21%-23%','23%-25%'
])

fig, axes = plt.subplots( figsize=(12, 6))
axes.set_title('Distribution of Charged Off Loans by Interest Rate')
sns.countplot( x='int_rate_groups', data=bin_10)
plt.show()

In [ ]:
bin_20 = df_3.copy()
bin_20['int_rate_groups'] = pd.cut(df_3_charged_off['int_rate'], bins=20,precision =0,labels=[
    '5%-6%','6%-7%','7%-8%','8%-9%','9%-10%','10%-11%','11%-12%','12%-13%','13%-14%','14%-15%','15%-16%','16%-17%','17%-18%','18%-19%','19%-20%','20%-21%','21%-22%','22%-23%','23%-24%','24%-25%'
])

fig, axes = plt.subplots( figsize=(12, 6))
axes.set_title('Distribution of Charged Off Loans by Interest Rate')
sns.countplot( x='int_rate_groups', data=bin_20)
plt.xticks(rotation=45)
plt.show()

In [ ]:
bin_4 = df_3_charged_off.copy()
bin_4['int_rate_groups'] = pd.cut(bin_4['int_rate'], bins=4,precision =0,labels=['5%-10%','10%-15%','15%-20%','20%-25%'])

bin_5 = df_3_charged_off.copy()
bin_5['int_rate_groups'] = pd.cut(bin_5['int_rate'], bins=5,precision =0,labels=['5%-9%','9%-13%','13%-17%','17%-21%','21%-24%'])

bin_10 = df_3_charged_off.copy()
bin_10['int_rate_groups'] = pd.cut(bin_10['int_rate'], bins=10,precision =0,labels=[
    '5%-7%','7%-9%','9%-11%','11%-13%','13%-15%','15%-17%','17%-19%','19%-21%','21%-23%','23%-25%'
])

bin_20 = df_3_charged_off.copy()
bin_20['int_rate_groups'] = pd.cut(bin_20['int_rate'], bins=20,precision =0,labels=[
    '5%-6%','6%-7%','7%-8%','8%-9%','9%-10%','10%-11%','11%-12%','12%-13%','13%-14%','14%-15%','15%-16%','16%-17%','17%-18%','18%-19%','19%-20%','20%-21%','21%-22%','22%-23%','23%-24%','24%-25%'
])

In [ ]:
fig, axes = plt.subplots(2,2, figsize=(12, 6))

axes[0][0].set_title('Distribution of Charged Off Loans by Interest Rate in 4 Bins')
axes[0][1].set_title('Distribution of Charged Off Loans by Interest Rate in 5 Bins')
axes[1][0].set_title('Distribution of Charged Off Loans by Interest Rate in 10 Bins')
axes[1][1].set_title('Distribution of Charged Off Loans by Interest Rate in 20 Bins')

sns.countplot( x='int_rate_groups', data=bin_4, ax=axes[0][0])
sns.countplot( x='int_rate_groups', data=bin_5, ax=axes[0][1])
sns.countplot( x='int_rate_groups', data=bin_10, ax=axes[1][0])
sns.countplot( x='int_rate_groups', data=bin_20, ax=axes[1][1])

# axes[0][0].set_xticklabels(axes[0][0].get_xticklabels(), rotation=45)
# axes[0][1].set_xticklabels(axes[0][1].get_xticklabels(), rotation=45)
axes[1][0].set_xticklabels(axes[1][0].get_xticklabels(), rotation=45)
axes[1][1].set_xticklabels(axes[1][1].get_xticklabels(), rotation=45)

plt.show()

In [ ]:
df_3['installment'].describe()

min_value = df_3['installment'].min()
max_value = df_3['installment'].max()
diff = max_value - min_value
bin_size = diff/12

print(min_value, max_value, diff, bin_size)

bin_edges = [0, 130, 260, 390, 520, 650, 780, 910, 1040, 1170, 1300, 1400]
bin_labels = [
    '0-130', '131-260', '261-390', '391-520',
    '521-650', '651-780', '781-910', '911-1040',
    '1041-1170', '1171-1300', '1301-1400'
]

df_3['installment_groups'] = pd.cut(df_3['installment'], bins=bin_edges, labels=bin_labels, right=False)
df_3_charged_off = df_3[ df_3['loan_status'] == 'Charged Off']
df_3_fully_paid = df_3[ df_3['loan_status'] == 'Fully Paid']

In [ ]:
fig, axes = plt.subplots( figsize=(12, 6))
axes.set_title('Distribution of Charged Off Loans by Installment')

sns.countplot( x='installment_groups', data=df_3_charged_off)
plt.show()

In [ ]:
df_3.info()

In [ ]:
fig, axes = plt.subplots( figsize=(12, 6))

# emp_length
axes.set_title('Distribution of Charged Off Loans by Employment Length')
sns.countplot( x='emp_length', data=df_3_charged_off)
plt.show()

In [ ]:
df_3['annual_inc'].describe( percentiles=[.25, .5, .75, .9, .95, .99])

In [ ]:
min_value = df_3['annual_inc'].min()  #4000.0
max_value = df_3['annual_inc'].max()  #234996.0
diff = max_value - min_value          #230996.0
bin_size = diff/12

print(min_value, max_value, diff, bin_size)

bin_edges = [
    0, 20000, 40000, 60000, 80000, 100000, 120000, 140000, 160000, 180000, 200000, 220000, 240000
]
bin_labels = [
    '0-20k', '20k-40k', '40k-60k', '60k-80k', '80k-100k', '100k-120k', '120k-140k', '140k-160k',
    '160k-180k', '180k-200k', '200k-220k', '220k-240k'
]


df_3['annual_inc_groups'] = pd.cut(df_3['annual_inc'], bins=bin_edges, labels=bin_labels, right=False)
df_3_charged_off = df_3[ df_3['loan_status'] == 'Charged Off']
df_3_fully_paid = df_3[ df_3['loan_status'] == 'Fully Paid']

In [ ]:
fig, axes = plt.subplots( figsize=(12, 6))

axes.set_title('Distribution of Charged Off Loans by Annual Income')
sns.countplot( x='annual_inc_groups', data=df_3_charged_off)
plt.show()

In [ ]:
df_3['dti'].describe( percentiles=[.25, .5, .75, .9, .95, .99])


min_value = df_3['dti'].min()  #0.0
max_value = df_3['dti'].max()  #29.99
diff = max_value - min_value   #29.99
bin_size = diff/12

print(min_value, max_value, diff, bin_size)

bin_edges = [
    0, 2.5, 5, 7.5, 10, 12.5, 15, 17.5, 20, 22.5, 25, 27.5, 30
]

bin_labels = [
    '0-2.5', '2.5-5', '5-7.5', '7.5-10', '10-12.5', '12.5-15', '15-17.5', '17.5-20',
    '20-22.5', '22.5-25', '25-27.5', '27.5-30'
]

df_3['dti_groups'] = pd.cut(df_3['dti'], bins=bin_edges, precision=0, labels=bin_labels, right=False)
df_3_charged_off = df_3[ df_3['loan_status'] == 'Charged Off']
df_3_fully_paid = df_3[ df_3['loan_status'] == 'Fully Paid']

In [ ]:
fig, axes = plt.subplots( figsize=(12, 6))

axes.set_title('Distribution of Charged Off Loans by Debt to Income Ratio')
sns.countplot( x='dti_groups', data=df_3_charged_off)
plt.show()

In [ ]:
print(df_3_charged_off['inq_last_6mths'].value_counts())

fig, axes = plt.subplots( figsize=(12, 6))
sns.countplot( x='inq_last_6mths', data=df_3_charged_off)
axes.set(yscale = 'log')
plt.show()

In [ ]:
df_3_charged_off['open_acc'].value_counts()

fig, axes = plt.subplots( figsize=(12, 6))
sns.countplot( x='open_acc', data=df_3_charged_off)
plt.show()

In [ ]:
df_3_charged_off['pub_rec'].value_counts()

## Bi-variate Analysis